In [ ]:
# 가상환경 구성 및 필요한 라이브러리 설치
# conda create -n LM
# conda install pip
# pip install pickle
# conda install gensim 
# conda install nltk
# conda install -c anaconda scikit-learn
# conda install pytorch==1.0.0 torchvision==0.2.1 cuda100 -c pytorch
# conda install matplotlib
# conda install pandas

In [1]:
# To ignore deprecated warnings
import warnings
warnings.simplefilter("ignore")
warnings.warn("deprecated", DeprecationWarning)
import os
import random
import numpy as np
from gensim.models import word2vec #  word2vec 관련 모델을 제공해주는 library  
import nltk
from nltk.corpus import movie_reviews # corpus 를 가져오는 용도 
from sklearn.manifold import TSNE # 시각화를 위한 TSNE 사용 
from sklearn.metrics import accuracy_score # accuracy score 를 계산하기 위한 metric 라이브러리 불러오기 
import matplotlib as mpl
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from torch.utils import data # pytorch data class 
import torch.nn as nn # pytorch neural network 불러오기 
import torch.nn.utils.rnn as rnn_utils # rnn utils
import torch

import pickle

#os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Chapter 1

# Word embedding (CBOW & Skip-Gram)



```
# This is formatted as code
```

![대체 텍스트](./figures/embedding.png)

![대체 텍스트](./figures/w2vec_1.png)

In [ ]:
# 파라메터값 지정
num_features = 300 # 임베딩 벡터 사이즈
negative = 10 # negative sampling할 단어 수
min_word_count = 10 # 단어빈도수가 10으로 제한
window = 5 # context window 사이즈
downsampling = 0.75 # 단어의 빈도수가 높은 단어에 대해 빈도수를 낮춤
epoch = 5 # epoch 수

In [ ]:
# preparing data
sentences = []
pos_data = open("./Data/lm_data/pos/train.txt").readlines()
neg_data = open("./Data/lm_data/neg/train.txt").readlines()
data_ = pos_data + neg_data
for line in data_:
    sentences.append(line.strip().split(' '))

In [ ]:
# skip-gram 모델 학습
skip_gram = word2vec.Word2Vec(sentences,
                              sg = 1, # skip-gram
                              negative=negative,
                              size=num_features, 
                              min_count=min_word_count,
                              window=window,
                              sample=downsampling,
                              iter=epoch)

# CBOW 모델 학습
CBOW = word2vec.Word2Vec(sentences,
                              sg = 0, # CBOW
                              negative=negative,
                              size=num_features, 
                              min_count=min_word_count,
                              window=window,
                              sample=downsampling,
                              iter=epoch)

##  Training result

In [ ]:
skip_gram.wv.most_similar("man") # skip-gram 모델을 이용하여 'man'에 가장 유사한 단어 추출

In [ ]:
CBOW.wv.most_similar("man") # CBOW 모델을 이용하여 'man'에 가장 유사한 단어 추출

In [ ]:
vector = skip_gram["king"] - skip_gram["man"] + skip_gram["woman"] # https://word2vec.kr/search/
skip_gram.wv.similar_by_vector(vector, topn=10, restrict_vocab=None)

In [ ]:
def render_TSNE(vocab, word_emb):
    """
    TSNE를 이용한 word2vec 시각화 
    args:
        vocab    - vocab list
        word_emb - word embeddings
    """
    tsne = TSNE(perplexity=15, n_components=2, init='pca', n_iter=3500, random_state=32) #TSNE 시각화를 위한 initialization
    _tsne = tsne.fit_transform(word_emb) # TSNE 시각화 적용
    x_coordinate = _tsne[:,0] # x 좌표
    y_coordinate = _tsne[:,1] # y 좌표

    # scatter plot initialization
    fig, ax = plt.subplots()
    fig.set_size_inches(40, 20)
    ax.scatter(x_coordinate, y_coordinate)

    for i, word in enumerate(random_vocab):
        ax.annotate(word,(x_coordinate[i], y_coordinate[i]), fontsize=30) # 각 scatter들에대해 단어 labeling

    plt.show()

In [ ]:
# Skip-gram 시각화(TSNE)
vocab = list(skip_gram.wv.vocab) # vocab list 불러오기
random_vocab = random.sample(vocab,k=100) #100개의 임의 단어를 랜덤 샘플링
word_emb = skip_gram[random_vocab] # 샘플링된 단어에 대해 학습된 임베딩 벡터 불러오기
render_TSNE(random_vocab, word_emb) # TSNE 시각화

In [ ]:
# CBOW 시각화(TSNE)
vocab = list(CBOW.wv.vocab) # vocab list 불러오기
random_vocab = random.sample(vocab,k=100) #100개의 임의 단어를 랜덤 샘플링
word_emb = skip_gram[random_vocab] # 샘플링된 단어에 대해 학습된 임베딩 벡터 불러오기
render_TSNE(random_vocab, word_emb) # TSNE 시각화

# Chapter2

# Lanaguage Model (RNN)

![대체 텍스트](./figures/lm_1.jpg)

In [90]:
def build_dict(seqs):
    num_skip_sent = 0
    word_count = 4
    vocab = ["<pad>","<s>","</s>","<unk>"]
    word2id = {"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3}
    id2word = {0: "<pad>", 1: "<s>", 2: "</s>", 3: "<unk>"}
    print("Building vocab and dict..")
    for line in seqs:
        words = line.strip().split(' ') # tokenized by space 
        for word in words:
            if word not in vocab:
                word_count += 1 # increment word_count
                vocab.append(word) # append new unique word
                index = word_count - 1 # word index (consider index 0)
                word2id[word] = index # word to index
                id2word[index] = word # index to word
    
    print("Number of unique words: %d" % len(vocab))
    print("Finised building vocab and dict!")

    return vocab, word2id, id2word

In [91]:
# Training corpus를 사용하여 vocab과 dict 구성
assert os.path.isfile("./Data/lm_data/pos/train.txt")
assert os.path.isfile("./Data/lm_data/neg/train.txt")
pos_seqs = open("./Data/lm_data/pos/train.txt").readlines()
neg_seqs = open("./Data/lm_data/neg/train.txt").readlines()
seqs = pos_seqs + neg_seqs # 긍정 corpus + 부정 corpus

In [92]:
count = 0
for i in range(len(seqs)):
    words = seqs[i].strip().split(' ')
    if len(words) >= 0:
        count+=1
print(count)
print(len(seqs))

62966
62966


In [93]:
# Building vocab or loading existing vocab
path = "./Data/lm_id2word.pkl"
if os.path.isfile(path):
    with open("./Data/lm_id2word.pkl", "rb") as f:
        id2word = pickle.load(f)
    with open("./Data/lm_word2id.pkl", "rb") as f:
        word2id = pickle.load(f)
    with open("./Data/lm_vocab.pkl", "rb") as f:
        vocab = pickle.load(f)

else:
    vocab, word2id, id2word = build_dict(seqs)
    pickle.dump(vocab, open("./Data/lm_vocab.pkl", "wb" ))
    pickle.dump(word2id, open("./Data/lm_word2id.pkl", "wb" ))
    pickle.dump(id2word, open("./Data/lm_id2word.pkl", "wb" ))

In [94]:
# example of first 10 word2id
for index, word in enumerate(word2id.keys()):
    print({word: word2id[word]})
    if index == 9:
        break

{'<pad>': 0}
{'<s>': 1}
{'</s>': 2}
{'<unk>': 3}
{'films': 4}
{'adapted': 5}
{'from': 6}
{'comic': 7}
{'books': 8}
{'have': 9}


In [95]:
# example of first 10 id2word
for index, id in enumerate(id2word.keys()):
    print({id: id2word[id]})
    if index == 9:
        break

{0: '<pad>'}
{1: '<s>'}
{2: '</s>'}
{3: '<unk>'}
{4: 'films'}
{5: 'adapted'}
{6: 'from'}
{7: 'comic'}
{8: 'books'}
{9: 'have'}


In [96]:
class LM_Dataset(data.Dataset):

    def __init__(self, pos_path, neg_path , word2id):
        
        #(add+)
        self.pos_seqs = open(pos_path).readlines()
        self.pos_labels = [0] * len(self.pos_seqs)
        self.neg_seqs = open(neg_path).readlines()
        self.neg_labels = [1] * len(self.neg_seqs)
        self.seqs = self.pos_seqs + self.neg_seqs # concat pos_seqs and neg_seqs
        self.labels = self.pos_labels + self.neg_labels
        
        self.word2id = word2id


    def __getitem__(self, index): # 가장 중요한 part Dataset 을 상속한 클래스는 이 부분을 overriding 해줘야함. 
        """Returns one data pair (source and sentiment)."""
        seqs = self.seqs[index]
        seqs = self.process(seqs, self.word2id)
        sentiment_labels = self.labels[index]
        return seqs, sentiment_labels

    def __len__(self): # 필수적인 클래스 이 부분을 무조건 overriding 해야함 
        return len(self.seqs) # 데이터 갯수 return 
                    
    def process(self, seq, word2id):
        sequence = []
#         sequence.append(word2id["<s>"])
        words = seq.strip().split(' ')
#         if len(words) <= 50: # max seq_len = 100
        for word in words:
            if word in word2id:
                sequence.append(word2id[word]) # 
            else:
                sequence.append(3) # replace by <unk> token
#         sequence.append(word2id["</s>"])
        sequence = torch.Tensor(sequence)
        return sequence

### Batch 를 구성해보겠습니다. 
Q : 우리의 문장에서 항상 단어의 갯수는 일정한가요?

문제점: Tensor로 변환하기 위해서는 추가 처리 필요 . eg. Sentence1 : W1 , W2, W3 ,W4 / Sentence : W1 , W2 ,W3 ->  2*3이 맞나요? 2*4가 맞나요? 

해결법 : Batch * max_length  Tensor로 변환합니다. eg. 2*4의 경우 2 번째 문장의 마지막에 PAD_token을 처리해줍니다. Index 보통 : 0 

In [97]:
def pad_tensor(vec, pad, value=0, dim=0):
    """
    pad token으로 채우는 용도 
    args:
        vec - tensor to pad
        pad - the size to pad to
        dim - dimension to pad
    return:
        a new tensor padded to 'pad' in dimension 'dim'
    """
    pad_size = pad - vec.shape[0]

    if len(vec.shape) == 2:
        zeros = torch.ones((pad_size, vec.shape[-1])) * value
    elif len(vec.shape) == 1:
        zeros = torch.ones((pad_size,)) * value
    else:
        raise NotImplementedError
    return torch.cat([torch.Tensor(vec), zeros], dim=dim)

![대체 텍스트](./figures/sorted.jpg)

In [98]:
temp = lambda x: x*x
temp(2)

4

In [99]:
def collate_fn(batch, values=(0, 0), dim=0):
    """
    데이터 로더에 들어가기전에 batch화 할 때 거치는 함수 
    args:
        batch - list of (tensor, label)
    reutrn:
        xs - a tensor of all examples in 'batch' after padding
        ys - a LongTensor of all labels in batch
        ws - a tensor of sequence lengths
    """
    sequence_lengths = torch.Tensor([int(x[0].shape[dim]) for x in batch]) # 각 batch 마다 길이를 얻어내고 
    sequence_lengths, xids = sequence_lengths.sort(descending=True) # 감소하는 순서로 정렬
    # find longest sequence (가장 긴 sequence의 길이를 구함 )
    # map(함수, 리스트) -> list
    # lambda 인자: 표현식
    max_len = max(map(lambda x: x[0].shape[dim], batch))
    # pad according to max_len (max length 만큼 padd를 추가 )
    batch = [(pad_tensor(x, pad=max_len, dim=dim), label) for (x, label) in batch]

    # stack all
    xs = torch.stack([x[0] for x in batch], dim=0)
    xs = xs[xids].contiguous() # decreasing order로 다시 나열
    
    labels = [x[1] for x in batch]
    labels = [labels[i] for i in xids] # decreasing order로 다시 나열
    
    return xs.long(), sequence_lengths.int(), torch.Tensor(labels)

![대체 텍스트](./figures/many_many.jpeg)



### Build Language model

![대체 텍스트](./figures/sorted.jpg)



![대체 텍스트](./figures/GRU.PNG)



![대체 텍스트](./figures/GRU_param.PNG)



![대체 텍스트](./figures/gru_output2.PNG)



### Example  pack padded

In [100]:
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
b

tensor([[1, 2, 3],
        [3, 4, 0]])

In [101]:
pack_b=torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])
pack_b

PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]))

In [102]:
rnn_utils.pad_packed_sequence(pack_b,batch_first=True,padding_value=0.)

(tensor([[1, 2, 3],
         [3, 4, 0]]),
 tensor([3, 2]))

## Model

In [103]:
class LM(nn.Module):
    def __init__(self, config, vocab_size):
        super(LM, self).__init__()
        self.vocab_size = vocab_size # get vocabulary size 
        self.embedding = nn.Embedding(self.vocab_size, config["embedding_size"])  # initialize embedding layer 
        self.rnn = nn.RNN(input_size=config["embedding_size"], hidden_size=config["hidden_size"], 
                          dropout=config["dropout"], batch_first=True) # defined rnn layer 
        self.outputs2vocab = nn.Linear(config["hidden_size"], self.vocab_size) # linear layer to predict the sentiment
    def forward(self,input, lengths):
        batch_size = input.size(0) # get batch_size
        input_embedding = self.embedding(input) # embedding layer  (B,L) -> (B,L ,hidden)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, lengths, batch_first=True) # pack sequence 
        outputs, _ = self.rnn(packed_input)   # rnn 
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True, padding_value=0.)[0] # packed sequence 
        padded_outputs = padded_outputs.contiguous()
        outputs = self.outputs2vocab(padded_outputs) 
        logits = outputs.contiguous().view(-1, self.vocab_size)
        return logits

In [104]:
config = {
  "epochs": 10,
  "hidden_size": 128,
  "dropout": 0.5,
  "batch_size": 32,
  "embedding_size": 256,
  "learning_rate": 0.001,
}

In [105]:
train_dataset = LM_Dataset("./Data/lm_data/pos/train.txt", "./Data/lm_data/neg/train.txt", word2id)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=True,
                                          collate_fn=collate_fn)
valid_dataset = LM_Dataset("./Data/lm_data/pos/valid.txt", "./Data/lm_data/neg/valid.txt", word2id)
valid_data_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=False,
                                          collate_fn=collate_fn)
test_dataset = LM_Dataset("./Data/lm_data/pos/test.txt", "./Data/lm_data/neg/test.txt", word2id)
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=False,
                                          collate_fn=collate_fn)

![대체 텍스트](./figures/crossentropy.PNG)



![대체 텍스트](./figures/crossentropy_2.PNG)



![대체 텍스트](./figures/p13.jpg)



![대체 텍스트](./figures/p14.jpg)



![대체 텍스트](./figures/p17.jpg)

In [106]:
def lm_train(model, optimizer, train_loader, epoch,n_epochs):
    
    losses = []

    model.train() # train mode
    n_iter = 0
    for batch in train_loader:
        input, input_lengths, _ = batch 
        output = model(input[:,:-1].contiguous().to(device),(input_lengths-1).to(device))
        loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)
        target = input[:,1:].contiguous().view(-1) 
        loss = loss_fn(output,target.to(device))
        losses.append(loss.item())
        # Reset gradients
        optimizer.zero_grad()
        # Compute gradients
        loss.backward()
        optimizer.step() # step optimzier 
        n_iter+=1 # count number of trained sentences
        if n_iter % 100 == 0: # print loss info for every 100 iteration steps
            print ('\n [{}] current_iter_loss= {:05.3f}'.format(n_iter,loss))
  
    print ('\n Epoch({}/{}) avg_loss= {:05.3f}'.format(epoch+1,n_epochs,np.mean(losses)))
    
    return model, optimizer

In [107]:
def idx2word(idx, i2w, pad_idx):
    """
    index로 이루어진 문장을 받아,
    word 문장으로 전환
    """

    sent_str = [str()]*len(idx)

    for i, sent in enumerate(idx):

        for word_id in sent:

            if word_id == pad_idx:
                break
            sent_str[i] += i2w[int(word_id)] + " "

        sent_str[i] = sent_str[i].strip()


    return sent_str[1]

In [108]:
def lm_eval(model, loader):
    losses = []

    model.eval() # valid mode
    n_iter = 0
 
    for batch in loader:
        input, input_lengths, _ = batch
        output = model(input[:,:-1].contiguous().to(device),(input_lengths-1).to(device)) # x vocab_size
        loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)
        target = input[:,1:].contiguous().view(-1)
        loss = loss_fn(output,target.to(device))
        losses.append(loss.item())
    prediction = torch.topk(output, 1)[1].squeeze(1).view(input.shape[0],-1)
    print("\n[Output sentence:] ")
    print(idx2word(prediction, id2word, 0) + '\n')
    print("[Target sentence:] ")
    print(idx2word(input[:,1:], id2word, 0))
    
    return losses

In [109]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LM(config, vocab_size=len(vocab))
optimizer = torch.optim.Adam(model.parameters(), lr=config.get("learning_rate", .001))

# Train model
for epoch in range(config["epochs"]):
    model,optimizer  = lm_train(model.to(device),optimizer, train_data_loader, epoch,config["epochs"]) # train
    valid_loss = lm_eval(model.to(device), valid_data_loader) # valid
    print ('\n              avg_valid loss= {:05.3f}'.format(np.mean(valid_loss)))
    test_loss = lm_eval(model.to(device), test_data_loader) # test
    print ('\n              avg_test loss= {:05.3f}'.format(np.mean(test_loss)))

RuntimeError: Length of all samples has to be greater than 0, but found an element in 'lengths' that is <= 0

### 더 큰 hidden_size

In [ ]:
config = {
  "epochs": 5,
  "hidden_size": 128,
  "dropout": 0.2,
  "batch_size": 32,
  "embedding_size": 128,
  "learning_rate": 0.001,
}

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LM(config, vocab_size=len(vocab))
optimizer = torch.optim.Adam(model.parameters(), lr=config.get("learning_rate", .001))

# Train model
for epoch in range(config["epochs"]):
    model,optimizer  = lm_train(model.to(device),optimizer, train_data_loader, epoch,config["epochs"]) # train
    valid_loss = lm_eval(model.to(device), valid_data_loader) # valid
    print ('\n              avg_valid loss= {:05.3f}'.format(np.mean(valid_loss)))
    test_loss = lm_eval(model.to(device), test_data_loader) # test
    print ('\n              avg_test loss= {:05.3f}'.format(np.mean(test_loss)))

### GRU로 변경하기 

In [ ]:
class LM(nn.Module):
    def __init__(self, config, vocab_size):
        super(LM, self).__init__()
        self.vocab_size = vocab_size # get vocabulary size 
        self.embedding = nn.Embedding(self.vocab_size, config["embedding_size"])  # initialize embedding layer 
        self.rnn = nn.GRU(input_size=config["embedding_size"], hidden_size=config["hidden_size"], 
                          dropout=config["dropout"], batch_first=True) # defined rnn layer 
        self.outputs2vocab = nn.Linear(config["hidden_size"], self.vocab_size) # linear layer to predict the sentiment
    def forward(self,input, lengths):
        batch_size = input.size(0) # get batch_size
        input_embedding = self.embedding(input) # embedding layer  (B,L) -> (B,L ,hidden)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, lengths, batch_first=True) # pack sequence 
        outputs, _ = self.rnn(packed_input)   # rnn 
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True, padding_value=0.)[0] # packed sequence 
        padded_outputs = padded_outputs.contiguous()
        outputs = self.outputs2vocab(padded_outputs) 
        logits = outputs.contiguous().view(-1, self.vocab_size)
        return logits

In [ ]:
config = {
  "epochs": 5,
  "hidden_size": 128,
  "dropout": 0.2,
  "batch_size": 32,
  "embedding_size": 128,
  "learning_rate": 0.001,
}

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LM(config, vocab_size=len(vocab))
optimizer = torch.optim.Adam(model.parameters(), lr=config.get("learning_rate", .001))

# Train model
for epoch in range(config["epochs"]):
    model,optimizer  = lm_train(model.to(device),optimizer, train_data_loader, epoch,config["epochs"]) # train
    valid_loss = lm_eval(model.to(device), valid_data_loader) # valid
    print ('\n              avg_valid loss= {:05.3f}'.format(np.mean(valid_loss)))
    test_loss = lm_eval(model.to(device), test_data_loader) # test
    print ('\n              avg_test loss= {:05.3f}'.format(np.mean(test_loss)))

# Chatper 3 

# Sentiment Classfication

![대체 텍스트](./figures/many_many.jpeg)



![대체 텍스트](./figures/sentence_clas.png)



In [ ]:
class Sentiment_Dataset(data.Dataset):

    def __init__(self, path , word2id):
        self.seqs = open(path).readlines()
        self.word2id = word2id

    def __getitem__(self, index):
        """Returns one data pair (source and sentiment)."""
        seqs = self.seqs[index]
        seqs, label = self.process(seqs, self.word2id)
        return seqs, label

    def __len__(self):
        return len(self.seqs)
                    
    def process(self, seq, word2id):
        label = 0 # default label "pos"
        sequence = []
        sequence.append(word2id["<s>"])
        words = seq.strip().split(' ')
        for i in range(0, len(words)-1):
            current_word = words[i]
            if "negative" == words[len(words)-1]: #if label is "neg", then 1
                label = 1
            if current_word in word2id:
                sequence.append(word2id[current_word])
            else:
                sequence.append(3) # replace by <unk> token
        sequence.append(word2id["</s>"])
        sequence = torch.Tensor(sequence)
        return sequence, label

In [ ]:
class Sentiment_Classification(nn.Module):
    """
    IMDb 영화 리뷰 감성분류 모델
    """
    def __init__(self, config, vocab_size):
        """
        args:
            config     - hyperparameters
            vocab_size - vocab_size
        return:
            None
        """
        super(Sentiment_Classification, self).__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(self.vocab_size, config["embedding_size"])
        self.rnn = nn.RNN(input_size=config["embedding_size"], hidden_size=config["hidden_size"], 
                          num_layers=config["num_layers"],dropout=config["dropout"],bidirectional = config['bidirectional'] , batch_first=True)
        self.bidirectional = config['bidirectional']
        self.num_layers = config['num_layers']
        if self.bidirectional:
            self.hidden_size = 2*config['hidden_size']
        else:
            self.hidden_size = config['hidden_size']
        self.outputs = nn.Sequential(
                            nn.Linear(self.hidden_size, self.hidden_size),
                            nn.LeakyReLU(0.2),
                            nn.Linear(self.hidden_size, self.hidden_size),
                            nn.LeakyReLU(0.2),
                            nn.Linear(self.hidden_size, 1),
                            nn.Sigmoid()
                        )
    def forward(self,input, lengths):
        """
        args:
            input   - setences
            lengths - setence lengths
        return:
            outputs - positive or negative
        """
        batch_size = input.size(0)
        input_embedding = self.embedding(input)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, lengths, batch_first=True)
        _, hidden = self.rnn(packed_input)

        if self.bidirectional:
            batch_size = hidden.size()[1]
            hidden = hidden.view(self.num_layers,2,batch_size,-1)
            hidden = hidden.permute(2,0,1,3).contiguous().view(batch_size,self.num_layers,-1)
        else:
            hidden = hidden.permute(1,0,2)
            
        hidden = hidden[:,-1,:]
        outputs = self.outputs(hidden)
        outputs = outputs.squeeze(1)
        return outputs

![대체 텍스트](./figures/BCE1.PNG)



![대체 텍스트](./figures/BCE2.PNG)



In [ ]:
def sentiment_trainer(model, optimizer, loaders, epoch,n_epochs):
    """
    args:
        model     - sentiment model
        optimizer - adam
        loaders   - valid, train loaders
    return:
        model, optimizer
    """
    losses = []
    total_accuracy = []
    val_losses = []
    val_total_accuracy = []
    n_iter = 0
    for split in loaders.keys():
        if split == "train":
            for batch in loaders[split]:
                model.train() # train mode
                input, input_lengths, target_label = batch
                predict_label = model(input.to(device),input_lengths.to(device))
                loss_fn = nn.BCELoss().to(device)
                loss = loss_fn(predict_label,target_label.to(device))
                losses.append(loss.item())
                # Calculate accuracy
                x_acc = predict_label.round().cpu().detach().numpy()
                y_acc = target_label.cpu().detach().numpy()
                accuracy = accuracy_score(x_acc,y_acc)
                total_accuracy.append(accuracy)
                # Reset gradients
                optimizer.zero_grad()
                # Compute gradients
                loss.backward()
                optimizer.step()
                n_iter+=1 # count number of trained sentences
                if n_iter % 100 == 0: # print loss only if it's training stage
                    print ('\n [{}] current_iter_loss= {:05.3f} acc= {:05.3f}'.format(n_iter,loss,accuracy))
                
        elif split == "valid":
            model.eval() # eval mode
            for batch in loaders[split]:
                input, input_lengths, target_label = batch
                predict_label = model(input.to(device),input_lengths.to(device))
                loss_fn = nn.BCELoss().to(device)
                loss = loss_fn(predict_label,target_label.to(device))
                val_losses.append(loss.item())
                # Calculate accuracy
                x_acc = predict_label.round().cpu().detach().numpy()
                y_acc = target_label.cpu().detach().numpy()
                accuracy = accuracy_score(x_acc,y_acc)
                val_total_accuracy.append(accuracy)
  
    print ('\n Epoch({}/{}) avg train_loss= {:05.3f} train_acc= {:05.3f} val_loss= {:05.3f} val_acc= {:05.3f}'
           .format(
               epoch+1,n_epochs,np.mean(losses), np.mean(total_accuracy), 
                   np.mean(val_losses), np.mean(val_total_accuracy)
              )
          )
    
    return model, optimizer

In [ ]:
def idx2word(idx, i2w, pad_idx):
    """
    index로 이루어진 문장을 받아,
    word 문장으로 전환
    """

    sent_str = [str()]*len(idx)

    for i, sent in enumerate(idx):

        for word_id in sent:

            if word_id == pad_idx:
                break
            sent_str[i] += i2w[int(word_id)] + " "

        sent_str[i] = sent_str[i].strip()


    return sent_str[0]

In [ ]:
def evaluate(model, loader, epoch,n_epochs, id2word):
    """
    각 epoch 마다 model에 대한 test및, prediction결과 추출
    """
    losses = []
    total_accuracy = []
    model.eval()
    for batch in loader:
        input, input_lengths, target_label = batch
        predict_label = model(input.to(device),input_lengths.to(device))
        loss_fn = nn.BCELoss().to(device)
        loss = loss_fn(predict_label,target_label.to(device))
        losses.append(loss.item())
        # Calculate accuracy
        x_acc = predict_label.round().cpu().detach().numpy()
        y_acc = target_label.cpu().detach().numpy()
        accuracy = accuracy_score(x_acc,y_acc)
        total_accuracy.append(accuracy)

    print ('\n Epoch({}/{}) avg test_loss= {:05.3f} test_acc= {:05.3f}'
        .format(epoch+1,n_epochs,np.mean(losses), np.mean(total_accuracy))
    )
    
    # test inference
    print (idx2word(input, id2word, 0))
    predict_label = model(input.to(device),input_lengths.to(device))
    ans = {0: 'positive', 1:'negative'}
    print('\n Pred: {}, Ans: {}'
          .format(ans[predict_label[0].round().item()], ans[target_label[0].item()])
         )


In [ ]:
seqs = open("./Data/lm_data/new_IMDb/train.tok").readlines()
vocab, word2id, id2word = build_dict(seqs)

In [ ]:
# Building vocab or loading existing vocab
path = "./Data/sentimen_id2word.pkl"
if os.path.isfile(path):
    with open("./Data/sentimen_id2word.pkl", "rb") as f:
        id2word = pickle.load(f)
    with open("./Data/sentimen_word2id.pkl", "rb") as f:
        word2id = pickle.load(f)
    with open("./Data/sentiment_vocab.pkl", "rb") as f:
        vocab = pickle.load(f)

else: # file does not exist
    vocab, word2id, id2word = build_dict(seqs)
    pickle.dump(vocab, open("./Data/sentiment_vocab.pkl", "wb" ))
    pickle.dump(word2id, open("./Data/sentiment_word2id.pkl", "wb" ))
    pickle.dump(id2word, open("./Data/sentiment_id2word.pkl", "wb" ))

In [ ]:
config = {
  "epochs": 5,
  "hidden_size": 64,
  "bidirectional": True,
  "num_layers": 3,
  "dropout": 0.2,
  "batch_size": 32,
  "embedding_size": 64,
  "learning_rate": 0.001,
}

In [ ]:
# constructing train dataset
train_dataset = Sentiment_Dataset("./Data/lm_data/new_IMDb/train.tok", word2id)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=True,
                                          collate_fn=collate_fn)
# constructing valid dataset
valid_dataset = Sentiment_Dataset("./Data/lm_data/new_IMDb/valid.tok", word2id)
valid_data_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=True,
                                          collate_fn=collate_fn)
# constructing test dataset
test_dataset = Sentiment_Dataset("./Data/lm_data/new_IMDb/test.tok", word2id)
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=True,
                                          collate_fn=collate_fn)

loaders = {"train": train_data_loader, "valid": valid_data_loader}

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Init sentiment model
model2 = Sentiment_Classification(config, vocab_size=len(vocab))
model2 = model2.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=config.get("learning_rate", .001))

# Train model
for epoch in range(config["epochs"]):
    model,optimizer  = sentiment_trainer(model2,optimizer, loaders, epoch,config["epochs"])
    evaluate(model, test_data_loader, epoch,config["epochs"], id2word)


## No Bidirectional 

In [ ]:
config = {
  "epochs": 5,
  "hidden_size": 64,
  "bidirectional": True,
  "num_layers": 2,
  "dropout": 0.2,
  "batch_size": 32,
  "embedding_size": 64,
  "learning_rate": 0.001,
}

In [ ]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model2 = Sentiment_Classification(config, vocab_size=len(vocab))
model2 = model2.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=config.get("learning_rate", .001))

# Train model
for epoch in range(config["epochs"]):
    model,optimizer  = sentiment_trainer(model2,optimizer, loaders, epoch,config["epochs"])
    evaluate(model, test_data_loader, epoch,config["epochs"], id2word)


### GRU Model 

In [ ]:
class Sentiment_Classification(nn.Module):

    def __init__(self, config, vocab_size):
        super(Sentiment_Classification, self).__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(self.vocab_size, config["embedding_size"])
        self.rnn = nn.GRU(input_size=config["embedding_size"], hidden_size=config["hidden_size"], 
                          num_layers=config["num_layers"],dropout=config["dropout"],bidirectional = config['bidirectional'] , batch_first=True)
        self.bidirectional = config['bidirectional']
        self.num_layers = config['num_layers']
        if self.bidirectional:
            self.hidden_size = 2*config['hidden_size']
        else:
            self.hidden_size = config['hidden_size']
        self.outputs = nn.Sequential(
                            nn.Linear(self.hidden_size, self.hidden_size),
                            nn.LeakyReLU(0.2),
                            nn.Linear(self.hidden_size, self.hidden_size),
                            nn.LeakyReLU(0.2),
                            nn.Linear(self.hidden_size, 1),
                            nn.Sigmoid()
                        )
    def forward(self,input, lengths):
        batch_size = input.size(0)
        input_embedding = self.embedding(input) 
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, lengths, batch_first=True)
        _, hidden = self.rnn(packed_input)
        
        if self.bidirectional:
            batch_size = hidden.size()[1]
            hidden = hidden.view(self.num_layers,2,batch_size,-1)
            hidden = hidden.permute(2,0,1,3).contiguous().view(batch_size,self.num_layers,-1)
        else:
            hidden = hidden.permute(1,0,2)
            
        hidden = hidden[:,-1,:]
        outputs = self.outputs(hidden)
        outputs = outputs.squeeze(1)
        return outputs

In [ ]:
config = {
  "epochs": 5,
  "hidden_size": 64,
  "bidirectional": True,
  "num_layers": 2,
  "dropout": 0.2,
  "batch_size": 32,
  "embedding_size": 64,
  "learning_rate": 0.001,
}

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model2 = Sentiment_Classification(config, vocab_size=len(vocab))
model2 = model2.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=config.get("learning_rate", .001))

# Train model
for epoch in range(config["epochs"]):
    model,optimizer  = sentiment_trainer(model2,optimizer, loaders, epoch,config["epochs"])
    evaluate(model, test_data_loader, epoch,config["epochs"], id2word)


## Let's Do it 

In [ ]:
config = {
  "epochs": 5,
  "hidden_size": 64,
  "bidirectional": True,
  "num_layers": 2,
  "dropout": 0.2,
  "batch_size": 32,
  "embedding_size": 64,
  "learning_rate": 0.001,
}

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model2 = Sentiment_Classification(config, vocab_size=len(vocab))
model2 = model2.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=config.get("learning_rate", .001))

# Train model
for epoch in range(config["epochs"]):
    model,optimizer  = sentiment_trainer(model2,optimizer, loaders, epoch,config["epochs"])
    evaluate(model, test_data_loader, epoch,config["epochs"], id2word)
